In [37]:
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as Soup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC


In [38]:
search= "臺灣大學"

In [39]:
browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# 開啟chromedriver
url = 'https://www.google.com.tw/maps?hl=zh-TW&tab=rl&authuser=0'
browser.get(url)

In [40]:
search_input = browser.find_elements(By.CLASS_NAME,'tactile-searchbox-input')[0]
search_input.send_keys(search)


In [41]:
search_click = browser.find_element(By.CLASS_NAME,'mL3xi')
search_click.click()

In [42]:
time.sleep(2.85)

In [43]:
soup = Soup(browser.page_source,"lxml")

### 爬取星級數

In [44]:
star = soup.find_all(class_="ceNzKf")[0]
star.get('aria-label')[:3]

'4.6'

In [45]:
star_section=[]
star_section.append(float((star.get('aria-label')[:3])))
print("學校星級：", star_section)

學校星級： [4.6]


In [46]:
school=pd.DataFrame({"學校名稱":search,"學校星級":star_section})
school

,學校名稱,學校星級
0,臺灣大學,4.6


### 爬取評論數

In [47]:
comment = soup.find_all(class_="F7nice")[0]
review = comment.text
review

'4.6(5,292)'

In [48]:
import re

In [49]:
review_num = []
# 使用正規表達式匹配括號內的數字
match = re.search(r'\(([\d,]+)\)', review)
reveiw_sum = match.group(1)
review_num.append(reveiw_sum)

# 檢查匹配是否成功
if match:
    # 提取匹配到的數字
    number_str = match.group(1)
    # 將字符串中的逗號去除
    number_str = number_str.replace(',', '')
    # 將字符串轉換為整數
    number = int(number_str)
    print(number)
else:
    print("未找到匹配的數字")

5292


In [50]:
school['學校總評論數']  = review_num

### 爬取評論

點擊進入更多評論

In [51]:
review_click = browser.find_elements(By.CLASS_NAME, "HHrUdb.fontTitleSmall.rqjGif")[0]
# 協助瀏覽器幫我們定位我們想要的位置
actions=ActionChains(browser)
# 自動幫我們滑到我們要的位置
actions.move_to_element(review_click).perform()
review_click.click()

點擊【排序】

In [52]:
menu_click = browser.find_elements(By.CLASS_NAME, "DVeyrd ")[2]
menu_click.click()

點擊【最新】

In [53]:
category_click = browser.find_elements(By.CLASS_NAME, "fxNQSd")[1]
category_click.click()


### 爬取多頁評論技巧(使用下拉式滾輪)

In [54]:
browser.maximize_window()
browser.execute_script("window.scrollBy(0, document.body.scrollHeight);")

In [55]:
scroll_time = 5 # 看要滑動幾次
for st in range(scroll_time):
    time.sleep(2)
    pane = browser.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]')
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
    print(st+1,'scroll')
    time.sleep(1)

1 scroll
2 scroll
3 scroll
4 scroll
5 scroll


### 選取評論元素和消費者名稱

In [56]:
time.sleep(2.85)
soup = Soup(browser.page_source,"lxml")

In [57]:
all_reviews = soup.find_all(class_ ='jftiEf fontBodyMedium')
len(all_reviews)

60

In [58]:
ar = all_reviews[0]

In [59]:
ar.find(class_ = 'd4r55').text

'梁靧'

In [60]:
ar.find(class_ = 'wiI7pd').text

'臺灣特有亞種「臺大球雀」的家，錯過和球雀在草地上滾來滾去的機會很可惜，一定要來玩喔：）'

In [61]:
### 爬取google評論及名稱
all_text_review=[]
all_title_review=[]
for i in range(len(all_reviews)):
    ar = all_reviews[i]
    all_title_review.append(ar.find(class_ = 'd4r55').text)
    if ar.find(class_ = 'wiI7pd') :
        all_text_review.append(ar.find(class_ = 'wiI7pd').text)
    else:
        all_text_review.append('')

In [62]:
df = pd.DataFrame({"評論者名稱":all_title_review,"評論內容":all_text_review})
df

,評論者名稱,評論內容
0,梁靧,臺灣特有亞種「臺大球雀」的家，錯過和球雀在草地上滾來滾去的機會很可惜，一定要來玩喔：）
1,YJ,
2,Ming Song Gu,生態資源豐富，親子怡人
3,吉他,運動活潑有朝氣。
4,TSMing WU,記憶中的世界
5,侯勝勛,
6,hung-yu shen,散步
7,Lisa Hsu,四季皆有不同風貌，很棒的地方。
8,陳鼎文,校園廣闊，適合散步帶，校區有UBike。
9,孫珮菱,
